In [55]:
import os
from shutil import rmtree
import collections
import pickle
import gzip

from tf.app import use

In [36]:
A = use("nena:clone", checkout="clone", hoist=globals())

In [37]:
C.levels.data

(('dialect', 269690.5, 539382, 539383),
 ('text', 4280.8015873015875, 713259, 713384),
 ('paragraph', 1536.6980056980058, 577912, 578262),
 ('line', 212.02083333333334, 575368, 577911),
 ('sentence', 32.28473095109834, 578263, 594969),
 ('subsentence', 21.991315692909854, 688732, 713258),
 ('inton', 14.989467541129391, 539384, 575367),
 ('stress', 5.752660992726264, 594970, 688731),
 ('word', 4.489304857342611, 713385, 833532),
 ('letter', 1, 1, 539381))

In [38]:
F.otype.all

('dialect',
 'text',
 'paragraph',
 'line',
 'sentence',
 'subsentence',
 'inton',
 'stress',
 'word',
 'letter')

In [39]:
F.otype.s("inton")

range(539384, 575368)

In [43]:
BASE = os.path.expanduser("~/github")
ORG = "CambridgeSemiticsLab"
REPO = "nena_explore"
REPO_DIR = f"{BASE}/{ORG}/{REPO}"
LOCAL_DIR = f"{REPO_DIR}/_local"
TYPES_DIR = f"{LOCAL_DIR}/nodeTypes"
EDGES_DIR = f"{LOCAL_DIR}/edges"

In [44]:
for outDir in ((TYPES_DIR, EDGES_DIR)):
    if os.path.exists(outDir):
        rmtree(outDir)
    os.makedirs(outDir, exist_ok=True)

In [45]:
def prepareType(nodeType):
    dataFile = f"{TYPES_DIR}/{nodeType}.txt"
    with open(dataFile, "w") as f:
        first = True
        for n in F.otype.s(nodeType):
            if first:
                f.write(f"@{n}\n")
            first = False
            text = T.text(n).replace("\n", "\\n").replace("\t", "\\t")
            f.write(f"{text}\n")

In [46]:
prepareType("stress")

In [47]:
for nodeType in F.otype.all[4:-1]:
    A.info(f"preparing {nodeType} ...")
    prepareType(nodeType)
    A.info("done")

 2m 31s preparing sentence ...
 2m 32s done
 2m 32s preparing subsentence ...
 2m 32s done
 2m 32s preparing inton ...
 2m 33s done
 2m 33s preparing stress ...
 2m 33s done
 2m 33s preparing word ...
 2m 34s done


In [60]:
def prepareInclusions():
    up = {}
    down = {}

    relevantTypes = F.otype.all[4:-1]
    relevantTypeSet = set(relevantTypes)

    for tp in F.otype.all[4:-1]:
        for n in F.otype.s(tp):
            ds = L.d(n, otype=relevantTypeSet)
            for d in ds:
                dtp = F.otype.v(d)
                down.setdefault(tp, {}).setdefault(n, {}).setdefault(dtp, []).append(d)
                up.setdefault(dtp, {}).setdefault(d, {}).setdefault(tp, []).append(n)

    for (data, fileName) in ((up, "up"), (down, "down")):
        with gzip.open(f"{EDGES_DIR}/{fileName}.pcl.gz", "wb", compresslevel=2) as f:
            pickle.dump(data, f)

In [61]:
A.info("preparing inclusions")
prepareInclusions()
A.info("done")

11m 44s preparing inclusions
11m 50s done
